In [1]:
import numpy as np
import sys
sys.path.append("policy/")
sys.path.append("arm/")
import egreedy
from bernoulli import BernoulliArm

In [2]:
arm1 = BernoulliArm(0.1)
arm2 = BernoulliArm(0.3)
arm3 = BernoulliArm(0.5)
arm4 = BernoulliArm(0.7)
arm5 = BernoulliArm(0.9)

arms = [arm1,arm2,arm3,arm4,arm5]

In [3]:
epsilon_greedy = egreedy.EpsilonGreedy(5,epsilon=0.1)
decay_egreedy = egreedy.DecayEpsilonGreedy(5)

In [4]:
total_reward = 0
for _ in range(100000):
    draw_arm = epsilon_greedy.select_arm()
    reward = arms[draw_arm].pull()
    total_reward += reward
    epsilon_greedy.update_states(draw_arm,reward)

In [5]:
total_draw = np.sum(epsilon_greedy.drawn_counts)
regret = np.random.binomial(total_draw,0.9) - total_reward
print("epsilon:",epsilon_greedy.epsilon)
print("expected reward:",epsilon_greedy.expected_reward)
print("drawn counts:",epsilon_greedy.drawn_counts)
print("regret:",regret)

epsilon: 0.1
expected reward: [0.10680576254346769, 0.3141907925961078, 0.49670552458185563, 0.6951466127401417, 0.8995855497177258]
drawn counts: [2013, 2107, 1973, 1978, 91929]
regret: 4078


In [6]:
total_reward = 0
for _ in range(100000):
    draw_arm = decay_egreedy.select_arm()
    reward = arms[draw_arm].pull()
    total_reward += reward
    decay_egreedy.update_states(draw_arm,reward)

In [7]:
total_draw = np.sum(decay_egreedy.drawn_counts)
regret = np.random.binomial(total_draw,0.9) - total_reward
print("epsilon:",decay_egreedy.get_epsilon())
print("expected reward:",decay_egreedy.expected_reward)
print("drawn counts:",decay_egreedy.drawn_counts)
print("regret:",regret)

epsilon: 0.000499750124938
expected reward: [0.17647058823529416, 0.29411764705882354, 0.5256410256410252, 0.5873015873015873, 0.90072500827292]
drawn counts: [68, 68, 78, 63, 99723]
regret: 145


In [8]:
import thompson

In [9]:
tsampling = thompson.BinaryThompsonSampling(5)

In [10]:
total_reward = 0
for _ in range(100000):
    i = tsampling.select_arm()
    reward = arms[i].pull()
    total_reward += reward
    tsampling.update_states(i,reward)

In [11]:
total_play = np.sum([tsampling.alpha,tsampling.beta]) - 10
regret = np.random.binomial(total_play,0.9) - total_reward
print("alpha:",tsampling.alpha)
print("beta:",tsampling.beta)
print("regret:",regret)

alpha: [1, 1, 6, 32, 89928]
beta: [5, 5, 8, 15, 10009]
regret: -222


In [12]:
total_regret = 0
for _ in range(100):
    epsilon_greedy = egreedy.EpsilonGreedy(5,epsilon=0.1)
    total_reward = 0
    for _ in range(10000):
        draw_arm = epsilon_greedy.select_arm()
        reward = arms[draw_arm].pull()
        total_reward += reward
        epsilon_greedy.update_states(draw_arm,reward)
    total_draw = np.sum(epsilon_greedy.drawn_counts)
    regret = np.random.binomial(total_draw,0.9) - total_reward
    total_regret += regret
print("EpsilonGreedy's average regret:",total_regret/100)

EpsilonGreedy's average regret: 440.11


In [13]:
total_regret = 0
for _ in range(100):
    decay_egreedy = egreedy.DecayEpsilonGreedy(5)
    total_reward = 0
    for _ in range(10000):
        draw_arm = decay_egreedy.select_arm()
        reward = arms[draw_arm].pull()
        total_reward += reward
        decay_egreedy.update_states(draw_arm,reward)
    total_draw = np.sum(decay_egreedy.drawn_counts)
    regret = np.random.binomial(total_draw,0.9) - total_reward
    total_regret += regret
print("DecayEpsilonGreedy's average regret:",total_regret/100)

DecayEpsilonGreedy's average regret: 110.9


In [14]:
total_regret = 0
for _ in range(100):
    tsampling = thompson.BinaryThompsonSampling(5)
    total_reward = 0
    for _ in range(10000):
        i = tsampling.select_arm()
        reward = arms[i].pull()
        total_reward += reward
        tsampling.update_states(i,reward)
    total_play = np.sum([tsampling.alpha,tsampling.beta]) - 10
    regret = np.random.binomial(total_play,0.9) - total_reward
    total_regret += regret
print("TS's average regret:",total_regret/100)

TS's average regret: 17.63


In [ ]:
import multiplay_thompson

In [ ]:
total_regret = 0
for _ in range(100):
    mp_ts = multiplay_thompson.BinaryMultiplayTS(5,2)
    total_reward = 0
    for _ in range(10000):
        selections = mp_ts.select_arm()
        for i in selections:
            reward = arms[i].pull()
            total_reward += reward
            mp_ts.update_states(i,reward)
    regret = np.random.binomial(10000,0.9)+np.random.binomial(10000,0.7) - total_reward
    total_regret += regret
print("MP-TS's average regret:",total_regret/100)